# Load Raw Dataset

In [1]:
import json
import os
import cv2

In [2]:
with open("data/raw/probe_labels.json") as file:
    raw_data = json.load(file)

# Convert Raw Dataset to YOLO format

In [3]:
def convert_to_yolo(bbox, img_width, img_height):
    x_min, y_min, box_width, box_height = bbox
    x_center = x_min + box_width / 2.0
    y_center = y_min + box_height / 2.0
    
    # Normalize
    x_center /= img_width
    y_center /= img_height
    box_width /= img_width
    box_height /= img_height

    return [x_center, y_center, box_width, box_height]

In [4]:
image_mapping = {image["id"]: image for image in raw_data["images"]}

class_id = 0

for annotation in raw_data["annotations"]:
    image_id = annotation["image_id"]
    bbox = annotation["bbox"]
    
    image_details = image_mapping[image_id]
    img_width = image_details["width"]
    img_height = image_details["height"]
    file_name = image_details["file_name"]
    
    yolo_bbox = convert_to_yolo(bbox, img_width, img_height)
    yolo_annotation = f"{class_id} {' '.join(map(str, yolo_bbox))}"
    
    txt_file_name = file_name.rsplit('.', 1)[0] + ".txt"
    directory = "data/yolo_annotation"
    with open(f"{directory}/{txt_file_name}", "w") as f:
        f.write(yolo_annotation)

# Draw Bounding Boxes around Probe using YOLO Dataset to make sure it is correct

In [5]:
def parse_yolo_annotation(annotation_file, image_width, image_height):
    with open(annotation_file, "r") as file:
        lines = file.readlines()

    bboxes = []
    for line in lines:
        data = line.strip().split()
        class_id, x_center, y_center, width, height = map(float, data)
        
        # Convert normalized values back to pixel values
        x_center *= image_width
        y_center *= image_height
        width *= image_width
        height *= image_height
        
        # Calculate top-left and bottom-right corners of the bounding box
        x_min = int(x_center - width / 2)
        y_min = int(y_center - height / 2)
        x_max = int(x_center + width / 2)
        y_max = int(y_center + height / 2)
        
        bboxes.append((class_id, x_min, y_min, x_max, y_max))
        
    return bboxes


def draw_bounding_boxes(image_path, annotation_path, output_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}")
        return

    image_height, image_width, _ = image.shape

    bboxes = parse_yolo_annotation(annotation_path, image_width, image_height)

    for bbox in bboxes:
        class_id, x_min, y_min, x_max, y_max = bbox
        color = (0, 255, 0)  # Green
        thickness = 2

        # Draw bounding box
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, thickness)

    os.makedirs(output_path, exist_ok=True)
    output_file = os.path.join(output_path, os.path.basename(image_path))
    cv2.imwrite(output_file, image)

In [6]:
image_folder = "data/raw/probe_images"
annotation_folder = "data/yolo_annotation"
output_folder = "data/images_with_bounding_boxes"

for image_file in os.listdir(image_folder):
    if image_file.endswith(".jpg"):
        image_path = os.path.join(image_folder, image_file)
        annotation_path = os.path.join(annotation_folder, image_file.rsplit('.', 1)[0] + ".txt")
        if os.path.exists(annotation_path):
            draw_bounding_boxes(image_path, annotation_path, output_folder)
        else:
            print(f"No annotation found for {image_file}")